# Calculation with splits from paper

## Data loading

In [1]:
import sys
sys.path.insert(0, "../..")
import os
import numpy as np
import scipy.io

In [2]:
# Load diagrams
data_path = "./pds.mat"
data_mat = scipy.io.loadmat(data_path)
data = data_mat["pds"]

print(f"Data shape: {data.shape}")
y = data_mat["labels"]
y = np.array(y).ravel()

X = data.reshape(-1)

Data shape: (1, 5700)


In [3]:
# Load splits, these are indexes of points in X, matlab index from 1 (so we do -1)
splits = scipy.io.loadmat('./presplited/3dshapes_splits.mat')
test_set = np.array( [arr[0][0] for arr in (splits['testSet'] - 1)] )
train_set = np.array( [arr[0][0] for arr in (splits['trainSet'] - 1)] )

tt_pairs = np.array([np.array(el) for el in zip(train_set, test_set)])

## First step: Model hyperparameter optimization

In [4]:
import joblib
import pandas as pd

from preprocessing import *
from persistent_bow import *
from visualization import *
import experiments.utils as ut

from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC, LinearSVC
from sklearn.mixture import GaussianMixture

from gudhi.representations.kernel_methods import SlicedWassersteinKernel
from gudhi.representations.vector_methods import PersistenceImage
from gudhi.representations.metrics import BottleneckDistance

#Fast hack to make SlicedWassersteinKernel scikit-compliant
setattr(SlicedWassersteinKernel, "get_params",
        lambda self, deep: {
            "bandwidth":self.bandwidth,
            "num_directions" : self.sw_.num_directions
        })

fold = StratifiedKFold(5, shuffle=True, random_state=42)
splits = np.array([split for split in fold.split(X, y)])

In [5]:
#Helper function, constructs final pipeline and returns girdsearch for it
def make_final_grid(estimator, param_grid, kernel="linear", *args, **kwargs ):
    new_param_grid = {f"Model__{name}" : values for name, values in param_grid.items()}
    new_param_grid["Predictor__C"] = [0.1, 1, 10]
    
    final_pipeline = Pipeline([
        ("Model", estimator),
        ("Predictor", SVC(kernel=kernel, max_iter=1e4))
    ])
    
    return GridSearchCV(final_pipeline, new_param_grid, cv = tt_pairs
                        , *args, **kwargs)

In [6]:
# Weight functions

def const(x):
    return 1

def linear(x):
    return x[1]

def pow2(x):
    return x[1]**2

In [7]:
#PBoW gridsearch
pbow_gridsearch = make_final_grid(
    estimator = PersistentBow(KMeans(7, n_init=1, max_iter=100, random_state=42),
                              sampler=RandomPDSampler(2500, random_state=42)),
    param_grid = {
        "cluster__n_clusters" : np.arange(10, 200, 10),
        "sampler__max_points" : np.arange(1000, 13000, 1000),
        "sampler__weight_function" : [const, linear, pow2]
    },
    n_jobs = -1
)

#SPBoW gridsearch
spbow_gridsearch = make_final_grid(
    estimator = StablePersistentBow(GaussianMixture(random_state=42),
                              sampler=RandomPDSampler(2500, random_state=42)),
    param_grid = {
        "mixture__n_components" : np.arange(10, 200, 10),
        "sampler__max_points" : np.arange(1000, 13000, 1000),
        "sampler__weight_function" : [const, linear, pow2]
    },
    n_jobs = -1
)

# PBOW with KMeans from VLFeat # TODO: This is Vl kmeans are 2 functions only
# We need to wrap it with something what fits fit, transform API
# pbow_on_stereoids = pbow_pipeline = Pipeline([
#     ("pbow",  PersistentBow(Vlkmeans(7,2), sampler=RandomPDSampler(2500)))
# ])


#SlicedWassersteinKernel gridsearch (without using it as kernel)
swk_gridsearch = make_final_grid(
    estimator = SlicedWassersteinKernel(),
    param_grid = {
        "bandwidth" : [0.05, 0.1, 0.25, 0.5, 1, 1.5, 2],
        "num_directions" : [5, 10, 15, 20, 25]
    },
    n_jobs = -1    
)

#SlicedWassersteinKernel gridsearch (using it as kernel)
swk_ker_gridsearch = make_final_grid(
    estimator = SlicedWassersteinKernel(),
    param_grid = {
        "bandwidth" : [0.05, 0.1, 0.25, 0.5, 1, 1.5, 2],
        "num_directions" : [5, 10, 15, 20, 25]
    },
    kernel="precomputed",
    n_jobs = -1
)

#BottleneckDistance gridsearch
bd_gridsearch = make_final_grid(
    estimator = BottleneckDistance(),
    param_grid = {
        "epsilon" : [1e-8, None]
    },
    n_jobs = -1  
)

#PersistenceImage gridsearch
pi_gridsearch = make_final_grid(
    estimator = PersistenceImage(),
    param_grid = {
        "bandwidth" : [0.1, 0.25, 0.5, 1, 1.5],
        "weight" : [const, linear, pow2],
        "resolution" : [(10,10), (20,20), (40,40), (50, 50)],
    },
    n_jobs = -1
)

models_to_test = {
    "PBoW" : pbow_gridsearch, 
    "SPboW" : spbow_gridsearch,
    "SWK" : swk_gridsearch, 
    "SWK_ker" : swk_ker_gridsearch,
    "Bottleneck" : bd_gridsearch,
    "PersistenceImage" : pi_gridsearch
}

In [8]:
for name, grid in models_to_test.items():
    print(name, end = " ")
    grid_path = f"precomputed/grid/{name}.dill"
    
    out = load(grid_path)
    if out:
        print("Loaded from file")
        models_to_test[name] = out
    else:
        grid.verbose = 10
        grid.fit(X, y)
        save(grid, grid_path)

PBoW
Loaded from file
SPboW
Loaded from file
SWK
Loaded from file
SWK_ker
Loaded from file
Bottleneck
Loaded from file
PersistenceImage
Loaded from file


In [9]:
#Grid computation
grid_path = f"./presplited/grid/"
cv_path = f"./presplited/cv/"

for filename in os.listdir(grid_path):
    name = os.path.splitext(filename)[0]
    grid = ut.load(os.path.join(grid_path, filename))
    results = ut.load(os.path.join(cv_path, filename))
    
    if not results:
        print("Computing", name)
        model = grid.best_estimator_
        results = cross_validate(model, X, y, cv=tt_pairs, n_jobs=-1, verbose=10)
        ut.save(results, os.path.join(cv_path, f"{name}.dill"))
    else:
        print(f"Loaded results: {name}")

Loaded results: SWK_ker
Loaded results: SPboW
Loaded results: PersistenceImage
Loaded results: PBoW
Computing Bottleneck
Skip Bottleneck because it hangs...
Loaded results: SWK


In [10]:
#Printing scores
base_path = f"./presplited/cv/"
for filename in os.listdir(base_path):
    path = os.path.join(base_path, filename)
    name = os.path.splitext(filename)[0]
    
    results = ut.load(path)
    print(name, "Mean scores:")
    df = pd.DataFrame(results)
    print(df.mean(), "\n")

PersistenceImage Mean scores:
fit_time      28.096989
score_time     8.174758
test_score     0.894489
dtype: float64 

SWK_ker Mean scores:
fit_time      280.212759
score_time    555.229747
test_score      0.951000
dtype: float64 

SPboW Mean scores:
fit_time      10.721233
score_time     3.950668
test_score     0.742857
dtype: float64 

PBoW Mean scores:
fit_time      10.206509
score_time     3.118982
test_score     0.853914
dtype: float64 

SWK Mean scores:
fit_time      277.187684
score_time    541.187843
test_score      0.948543
dtype: float64 



# Searching for difference between expperiment and paper

In [11]:
#PBoW gridsearch with different gamma, random state and params
pbow_gridsearch2 = GridSearchCV(
    Pipeline([
        ("Model", PersistentBow(KMeans(7, n_init=1, max_iter=300),
                              sampler=RandomPDSampler(2500))),
        ("Estimator", SVC(kernel="linear", max_iter=1e4, gamma='auto'))
    ]),
    {
        "Model__cluster__n_clusters" : np.arange(100, 250, 25),
        "Model__sampler__max_points" : np.arange(10000, 20000, 2500),
        "Model__sampler__weight_function" : [const, linear],
        "Estimator__C" : [0.1, 1, 10, 100, 1000]
    },
    n_jobs = -1,
    cv=tt_pairs
)

#PBoW gridsearch with different linear svc
pbow_gridsearch3 = GridSearchCV(
    Pipeline([
        ("Model", PersistentBow(KMeans(7, n_init=1, max_iter=300),
                              sampler=RandomPDSampler(2500))),
        ("Estimator", LinearSVC(max_iter=1e4, penalty="L1", loss="hinge"))
    ]),
    {
        "Model__cluster__n_clusters" : np.arange(60, 200, 25),
        "Model__sampler__max_points" : np.arange(1000, 20000, 2500),
        "Model__sampler__weight_function" : [const, linear],
        "Estimator__C" : [0.1, 1, 10],
        "Estimator__dual" : [True, False],
        "Estimator__loss" : ["hinge", "squared_hinge"],
        "Estimator__penalty" : ["l1", "l2"],
    },
    n_jobs = -1,
    cv=tt_pairs
)

#PBoW gridsearch without sampling
pbow_gridsearch4 = GridSearchCV(
    Pipeline([
        ("Model", PersistentBow(KMeans(7, n_init=10, max_iter=300),
                              sampler=RandomPDSampler(1000))),
            ("Estimator", SVC(kernel="linear", max_iter=1e4, gamma='auto'))
    ]),
    {
        "Model__cluster__n_clusters" : np.arange(100, 250, 25),
        "Model__sampler__max_points" : np.arange(10000, 20000, 2000),
        "Estimator__C" : [0.1, 1, 2, 5, 7, 10, 25]
    },
    n_jobs = -1,
    cv=tt_pairs
)

#PBoW with different kernels
pbow_gridsearch5 = GridSearchCV(
    Pipeline([
        ("Model", PersistentBow(KMeans(7, n_init=1, max_iter=300),
                              sampler=RandomPDSampler(2500))),
        ("Estimator", SVC(kernel="linear", max_iter=1e4, gamma='auto'))
    ]),
    {
        "Model__cluster__n_clusters" : np.arange(100, 250, 25),
        "Model__sampler__max_points" : np.arange(10000, 20000, 2000),
        "Model__sampler__weight_function" : [const, linear],
        "Estimator__C" : [0.1, 1, 10, 100, 1000],
        "Estimator__kernel" : ["rbf", "linear", "sigmoid", "poly"]
    },
    n_jobs = -1,
    cv=tt_pairs
)

#PBoW with different kernels on python-version splits
pbow_gridsearch6 = GridSearchCV(
    Pipeline([
        ("Model", PersistentBow(KMeans(7, n_init=1, max_iter=300),
                              sampler=RandomPDSampler(2500))),
        ("Estimator", SVC(kernel="linear", max_iter=1e4, gamma='auto'))
    ]),
    {
        "Model__cluster__n_clusters" : np.arange(100, 250, 25),
        "Model__sampler__max_points" : np.arange(10000, 20000, 2000),
        "Model__sampler__weight_function" : [const, linear],
        "Estimator__C" : [0.1, 1, 10],
        "Estimator__kernel" : ["rbf", "linear", "sigmoid", "poly"]
    },
    n_jobs = -1,
    cv=splits
)

In [12]:
pbow2_path = f"./presplited/secondary/pbow2.dill" 
grid = ut.load(pbow2_path)
if not grid:
    pbow_gridsearch2.verbose = 10
    pbow_gridsearch2.fit(X, y)
    ut.save(pbow_gridsearch2, pbow2_path)
else:
    pbow_gridsearch2 = grid

pbow3_path = f"./presplited/secondary/pbow3.dill" 
grid = ut.load(pbow3_path)
if not grid:
    pbow_gridsearch3.verbose = 10
    pbow_gridsearch3.fit(X, y)
    ut.save(pbow_gridsearch3, pbow3_path)
else:
    pbow_gridsearch3 = grid
    
pbow4_path = f"./presplited/secondary/pbow4.dill" 
grid = ut.load(pbow4_path)
if not grid:
    pbow_gridsearch4.verbose = 10
    pbow_gridsearch4.fit(X, y)
    ut.save(pbow_gridsearch4, pbow4_path)
else:
    pbow_gridsearch4 = grid
    
    
pbow5_path = f"./presplited/secondary/pbow5.dill" 
grid = ut.load(pbow5_path)
if not grid:
    pbow_gridsearch5.verbose = 10
    pbow_gridsearch5.fit(X, y)
    ut.save(pbow_gridsearch5, pbow5_path)
else:
    pbow_gridsearch5 = grid
    
pbow6_path = f"./presplited/secondary/pbow6.dill" 
grid = ut.load(pbow6_path)
if not grid:
    pbow_gridsearch6.verbose = 10
    pbow_gridsearch6.fit(X, y)
    ut.save(pbow_gridsearch6, pbow6_path)
else:
    pbow_gridsearch6 = grid

In [13]:
pbow_gridsearch2.best_score_

0.8588276588276589

In [14]:
pbow_gridsearch3.best_score_

0.8501228501228502

In [15]:
pbow_gridsearch4.best_score_

0.8548964548964548

In [16]:
pbow_gridsearch5.best_score_

0.8916812916812917

In [17]:
pbow_gridsearch6.best_score_

0.8978947368421053

In [23]:
pbow_gridsearch2.best_params_

{'Estimator__C': 1,
 'Model__cluster__n_clusters': 225,
 'Model__sampler__max_points': 12500,
 'Model__sampler__weight_function': <function __main__.linear(x)>}

In [36]:
pbow_gridsearch7 = GridSearchCV(
    Pipeline([
        ("Model", PersistentBow(KMeans(7, n_init=1, max_iter=300),
                              sampler=RandomPDSampler(2500))),
        ("Estimator", SVC(kernel="linear", max_iter=1e4, gamma="auto", tol=1e-4))
    ]),
    {
        "Model__cluster__n_clusters" : [225],
        "Model__sampler__max_points" : [12500],
        "Model__sampler__weight_function" : [const, linear],
        "Estimator__C" : [1],
        "Estimator__max_iter" : [1e4, 1e5, 1e6, 1e7, 1e8]
    },
    n_jobs = -1,
    cv=tt_pairs
)

pbow7_path = f"./presplited/secondary/pbow7.dill" 
grid = ut.load(pbow7_path)
if not grid:
    pbow_gridsearch7.verbose = 10
    pbow_gridsearch7.fit(X, y)
    ut.save(pbow_gridsearch7, pbow7_path)
else:
    pbow_gridsearch7 = grid

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 120 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  50 | elapsed:  2.0min remaining: 31.0min
[Parallel(n_jobs=-1)]: Done   9 out of  50 | elapsed:  2.0min remaining:  9.3min
[Parallel(n_jobs=-1)]: Done  15 out of  50 | elapsed:  2.2min remaining:  5.1min
[Parallel(n_jobs=-1)]: Done  21 out of  50 | elapsed:  2.4min remaining:  3.3min
[Parallel(n_jobs=-1)]: Done  27 out of  50 | elapsed:  2.5min remaining:  2.1min
[Parallel(n_jobs=-1)]: Done  33 out of  50 | elapsed:  2.9min remaining:  1.5min
[Parallel(n_jobs=-1)]: Done  39 out of  50 | elapsed:  3.0min remaining:   50.6s
[Parallel(n_jobs=-1)]: Done  45 out of  50 | elapsed:  3.0min remaining:   20.2s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  3.0min finished
/home/drenda/miniconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMa

In [34]:
model = PersistentBow(KMeans(225, n_init=1, max_iter=300),
                              sampler=RandomPDSampler(12500, const))

In [63]:
b = MinMaxScaler

NameError: name 'MinMaxScaler' is not defined

In [60]:
train_bows = model.fit_transform(X[tt_pairs[1][0]])
test_bows = model.transform(X[tt_pairs[1][1]])

In [61]:
c_svm = libsvm.svmutil.svm_train(y[tt_pairs[1][0]], train_bows, f"-s 0 -t 0 -c 1")

In [62]:
libsvm.svmutil.svm_predict(y[tt_pairs[1][1]], test_bows, c_svm);

Accuracy = 84.0997% (2396/2849) (classification)


In [42]:
estimator = SVC(kernel="linear", max_iter=1e6, gamma="auto", tol=1e-4, break_ties=True)

In [43]:
estimator.fit(train_bows, y[tt_pairs[0][0]])

SVC(C=1.0, break_ties=True, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=1000000.0, probability=False, random_state=None, shrinking=True,
    tol=0.0001, verbose=False)

In [45]:
estimator.score(test_bows, y[tt_pairs[0][1]])

0.8543348543348543